In [ ]:
!nvidia-smi

# environment

In [ ]:
!pip install gdown --quiet

### code

In [ ]:
%%bash
File_id="1AfRVKQtmHyoDd40PatqVOTzWiQaZyFs3"
File_name="benchmark"

gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"
unzip -q "${File_name}.zip" -d "${File_name}"
# rm -f "${File_name}.zip"

In [ ]:
%cd benchmark/

In [ ]:
%ll

### package

In [ ]:
!pip install -r requirements.txt --quiet # --q 讓他安靜

### data

In [ ]:
%%bash
File_id="1sp7D-Q_sqS5m3-4jKa98Tf6dU8KnR_rT"
gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"

In [ ]:
%%bash
folder="data/train/helas3_ctcf"

mkdir -p $folder
unzip -q "helas3_ctcf.zip" -d $folder

ls $folder

### Existing output

In [ ]:
%%bash
File_id="1p3usqMXPni7kGw6pQpAo_Yt23iGdB8TJ"
gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"

folder="output"

mkdir -p $folder
unzip -q "output.zip" -d $folder

# rename
# sour_folder="${folder}/24-5-11-basecnn-gm12878-ctcf-hyber-test-part2-1"
# tar_folder="${folder}/2024.07.07_cnn_hyper-test-basic-structure"

# mv $sour_folder $tar_folder

### run

In [ ]:
%%bash
#!/bin/bash
name="2024.07.07_cnn_hyper-test-basic-structure"

input_folder="data/train/helas3_ctcf"
out_folder="output/${name}"
input_format="onehot"

lr="0.001"
bp_range_ls="3000"

classifier_hidden_layer="3"
classifier_hidden_size="100"

extractor_hidden_size="100"
extractor_pool_type="max"
extractor_feature_aggregation="fc"
extractor_feature_agg_rate="0.5"

extractor_dropout="0.0 0.3 0.5"
classifier_dropout="0.0 0.3 0.5"

extractor_bn="True False"
classifier_bn="True False"

# 執行測試並顯示進度
total_tasks=36
completed_tasks=1

for a in $lr; do
    for b in $bp_range_ls; do
        for c in $classifier_hidden_layer; do
            for d in $classifier_hidden_size; do
                for e in $extractor_hidden_size; do
                    for f in $extractor_pool_type; do
                        for g in $extractor_feature_aggregation; do
                            for h in $extractor_feature_agg_rate; do
                                for i in $extractor_dropout; do
                                    for j in $classifier_dropout; do
                                        for k in $extractor_bn; do
                                            for l in $classifier_bn; do
                                                config="conf/base-CNN_${b}bp.yaml"
                                                input="${input_folder}/${b}bp.50ms.${input_format}/data.h5"
                                                out="${out_folder}/Bp${b}_Lr${a}_CHs${e}_Hs${d}_Hl${c}_Pt-${f}_Fa-${g}_Far${h}_ExtDrop${i}_ClfDrop${j}_ExtBn-${k}_ClfBn-${l}"
                                                echo
                                                echo ">>>>>>>>>>>>>>>>> Task <<<<<<<<<<<<<<<<<"
                                                echo "[$((${completed_tasks}))/${total_tasks}]"
                                                echo "Config: ${config}"
                                                echo "Input data: ${input}"
                                                echo "Out folder: ${out}"
                                                echo "<<<<<<<<<<<<<<<<< Task >>>>>>>>>>>>>>>>>"
                                                echo
                                                python train.py \
                                                    --config $config \
                                                    --input $input \
                                                    --output_folder $out \
                                                    --device gpu \
                                                    --eval_freq 1 \
                                                    --pin_memory_train True \
                                                    --train.learning_rate $a \
                                                    --model.extractor_hidden_size $e \
                                                    --model.extractor_output_dim $d \
                                                    --model.extractor_dropout $i \
                                                    --model.extractor_bn $k \
                                                    --model.extractor_pool_type $f \
                                                    --model.extractor_feature_aggregation $g \
                                                    --model.extractor_feature_agg_rate $h \
                                                    --model.classifier_input_dim $d \
                                                    --model.classifier_hidden_size $d \
                                                    --model.classifier_hidden_layer $c \
                                                    --model.classifier_dropout $j \
                                                    --model.classifier_bn $l
                                                # 檢查執行結果
                                                if [ $? -ne 0 ]; then
                                                    echo "Error: Training failed -> ${out_folder}"
                                                    exit 1
                                                fi

                                                ((completed_tasks++))
                                            done
                                        done
                                    done
                                done
                            done
                        done
                    done
                done
            done
        done
    done
done

# python code/helper_scripts/sort_exp_result.py --folder $out_folder


zip -r "output/${name}.zip" "output/${name}"


In [ ]:
# 一定要在這路徑底下才行
import os
os.chdir('/kaggle/working')

from IPython.display import FileLink
FileLink('benchmark/output/2024.07.07_cnn_hyper-test-basic-structure.zip')